In [307]:
# !pip install langchain
# !pip install langchain-openai
# !pip install neo4j
# !pip install --upgrade --quiet  google-search-results
# !pip install --quiet rogue
#!pip install -q -U google-generativeai
import numpy as np
!pip install --upgrade --quiet  langchain-google-genai

You should consider upgrading via the '/Users/swithana/git/ai-model-cards/venv/bin/python -m pip install --upgrade pip' command.


In [13]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain import OpenAI
from langchain.tools.retriever import create_retriever_tool
from langchain.prompts.prompt import PromptTemplate
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain_community.tools.google_scholar import GoogleScholarQueryRun
from langchain_community.utilities.google_scholar import GoogleScholarAPIWrapper
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama


from langchain_community.llms import Ollama
from langchain_google_vertexai import VertexAI
import google.generativeai as palm
from langchain_google_genai import GoogleGenerativeAI
import numpy as np

In [110]:
graph = Neo4jGraph(
    url="bolt://localhost:7689", username="neo4j", password="rootroot"
)

In [111]:
graph.refresh_schema()
print(graph.schema)

Node properties are the following:
Datasheet {Associated_tasks: STRING, Dataset_characteristics: STRING, target_variable: STRING, subject_area: STRING, source: STRING, name: STRING, license: STRING, missing_values: BOOLEAN, version: STRING, doi: STRING, external_id: STRING, categories: STRING, description: STRING, download_url: STRING, attribute_types: STRING, datapoints: INTEGER},ModelCard {input_type: STRING, author: STRING, output_data: STRING, name: STRING, keywords: STRING, external_id: STRING, categories: STRING, full_description: STRING, input_data: STRING, short_description: STRING, v_embedding: LIST, embedding: LIST, version: STRING},AIModel {train_accuracy: FLOAT, mean_squared_error: FLOAT, learning_rate: FLOAT, batch_size: INTEGER, name: STRING, license: STRING, version: STRING, num_epochs: INTEGER, test_accuracy: FLOAT, description: STRING, external_id: STRING, model_type: STRING, owner: STRING, foundational_model: STRING, framework: STRING, location: STRING},BiasAnalysis {

In [112]:
openai_api_key = ""
ser_api_key = ""
vertex_api_key = ""
palm.configure(api_key=vertex_api_key)

In [113]:
openai = OpenAI(temperature=0, openai_api_key=openai_api_key)
chat_gpt = ChatOpenAI(temperature=0.2, openai_api_key=openai_api_key, model="gpt-4")
chat_gpt35 = ChatOpenAI(temperature=0.2, openai_api_key=openai_api_key, model="gpt-3.5-turbo")
gemma = Ollama(model="gemma:7b")
codellama7b = Ollama(model="codellama:7b", num_predict=128, temperature=0.2)
codellama13b = Ollama(model="codellama:13b", num_predict=128, temperature=0.2)
# llama2 = Ollama(model="llama2:text")
mistral = ChatOllama(model="mistral", num_predict=128, temperature=0.2)
gemini = GoogleGenerativeAI(model="models/gemini-1.0-pro", google_api_key=vertex_api_key)
# codey = GoogleGenerativeAI(model="models/codechat-bison", google_api_key=vertex_api_key)

In [114]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.

DO NOT ALLOW DELETE, INSERT, UPDATE queries. In that case, say this is not allowed, contacting system administrator.

Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
This is important. if it mentions ID, that means external_id field.

When returning the modelcard or model information, return the external_id instead of the name.

If a model needs to refer to accuracy, it  usually refers to the accuracy information in the AI_MODEL associated with the model card. You can use a query like this to get that information linking the model card with technical information.  For example, to get the highest accuracy models:
match(mc:ModelCard)-[]->(ai:AIModel)
with mc.external_id as mcid, ai.test_accuracy as test_accuracy
order by test_accuracy DESC
limit 1
Return mcid, test_accuracy

But if it's deployment related, use the mean_accuracy in the deployment nodes instead.

Bias or fairness information about a model card is contained in the BIAS_ANALYSIS associated with the model card.

Devices does not have a name field. d.name is equal to d.device_id. Use that instead.
When querying for device, use the name of the device in the device_id field.

when returning model cards or models, remove the v_embedding field. Do not include that in results.

For devices, use the given name in the external_id field.

To getting similar to or closest to models, you can compare the model cards embedding field using cosine similarity.
Here's an example query to run to get the similar or close models. I've added <> for variables you can change.
MATCH (mc:ModelCard external_id: <id>)
                WITH mc.embedding AS given_embedding
                CALL db.index.vector.queryNodes('embedding', <num of results>, given_embedding) yield node, score
                RETURN score, node.external_id AS id

To get the model with the best fairness you can use the following query.

match(n:ModelCard)-[]->(f:BiasAnalysis)
with n, ABS(f.demographic_parity_diff) as dem, ABS(f.equal_odds_difference) as eq
ORDER BY dem + eq ASC
Limit 1
Return n.external_id, dem, eq

When calculating lowest values for demographic_parity_difference or equal_odds_difference, refer to the absolute value.

If prompted about framework or license, refer to the corresponding AI_MODEL to get the information.

if you can't find the answer or do not know the answer return "it's not available in the database.
ONLY RETURN THE CYPHER QUERY!

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)


In [115]:
QA_TEMPLATE = """Task:Generate a response to the question only using the provided context.

The context contains information retrieved from running a query corresponding to the question.
Generate a text based answer to the question imitating a human answering. Limit the response to one sentence.

The context is:
{context}

The question is:
{question}"""

QA_GEN_TEMPLATE = PromptTemplate(
    input_variables=["context", "question"], template=QA_TEMPLATE
)


In [ ]:
neo4jQA = GraphCypherQAChain.from_llm(
    llm=chat_gpt,
    # cypher_llm=codellama13b,
    # qa_llm=codellama13b,
    graph=graph,
    verbose=False,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    # qa_prompt=QA_GEN_TEMPLATE,
    validate_cypher=True,
)

In [120]:
result = neo4jQA.run("How many models have been created using the UCI Adult Dataset?")
print(result)

2000 models have been created using the UCI Adult Dataset.


In [107]:
# evaluation
import time

results = []
times = []
i = 1
with open("/Users/swithana/git/ai-model-cards/llm/evaluation/questions.txt", "r") as file:
    for line in file:
        print("Question: {}\t:{}".format(i, line))
        i = i+1
        try:
            query_time = time.time()
            result = neo4jQA.run(line)
            query_total_time = time.time() - query_time
            times.append(query_total_time)
            print(result)
        except Exception as e:
            results.append("ERROR")
            print("ERROR")
            continue
        # results.append(result['result'])
        results.append(result)

print(np.average(times))
with open("/Users/swithana/git/ai-model-cards/llm/evaluation/codellama13b-low-temp.txt", "w") as file:
        # Iterate through each line in the list
        for line in results:
            # Write the line to the file
            file.write(line + "\n")
        file.write("Time per query: {}".format(times))

Question: 1	:Who's the author of the model e835e0f3-3c13-4ab8-9b17-a8c5ea8dc788 ?



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (m:ModelCard {external_id: 'e835e0f3-3c13-4ab8-9b17-a8c5ea8dc788'})-[:AI_MODEL]->(ai:AIModel) RETURN ai.author AS author
Full Context:
[{'author': None}]

> Finished chain.
 The author of the model e835e0f3-3c13-4ab8-9b17-a8c5ea8dc788 is not specified in the context.
Question: 2	:What's the framework used for the model e835e0f3-3c13-4ab8-9b17-a8c5ea8dc788 ?



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (m:ModelCard)-[:AI_MODEL]->(ai:AIModel)
WHERE m.external_id = 'e835e0f3-3c13-4ab8-9b17-a8c5ea8dc788'
RETURN ai.framework AS framework
Full Context:
[{'framework': 'tensorflow'}]

> Finished chain.
 The framework used for the model e835e0f3-3c13-4ab8-9b17-a8c5ea8dc788 is TensorFlow.
Question: 3	:How many models have been created using the UCI Adult Dataset?



> Entering new GraphCypherQAChain chain...
Generated C

In [15]:
template = """Your job is to answer the question below.
{question}
"""
prompt_template = PromptTemplate(input_variables=["question"], template=template)

llm_chain = LLMChain(
    llm=chat_gpt,
    prompt=prompt_template,
    verbose=True,
)


In [16]:
scholar_tool = GoogleScholarQueryRun(api_wrapper=GoogleScholarAPIWrapper(serp_api_key=ser_api_key))

In [17]:
tools = [
    Tool(
        name="llm",
        description="Use this as the last resort if none of the other tools provide an output.",
        func=llm_chain.run,
    ),
    Tool(
        name = 'ICICLE_MC',
        func = neo4jQA.invoke,
        description = ( 'Use this to query about models, datasheet, deployment and their analysis. This is also referred to as ICICLE database, database, knowledge graph. If the query has a prefix IMC, parse the original query as is to this tool and return the result. Dont use any other tool after. DO NOT ALLOW DELETE, INSERT, UPDATE. say you are contacting administrator in that case. '
        )
    ),
    Tool(
        name="google_scholar",
        description="Use this to search google scholar for research citations, papers, research papers results. If the prefix for query is GS: only run it using this tool. Don't run any subsequent tools",
        func=scholar_tool.run,
    )
]

memory = ConversationBufferWindowMemory(k=2)

agent = initialize_agent(
    tools=tools,
    llm=openai,
    verbose=True,
    memory=memory
)

/Users/swithana/git/icicle_model_card/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [18]:
query="What is the ICICLE project?"
agent(query)

/Users/swithana/git/icicle_model_card/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use ICICLE_MC to query about the project.
Action: ICICLE_MC
Action Input: What is the ICICLE project?

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:ModelCard) WHERE m.name = 'ICICLE project' RETURN m.short_description, m.full_description, m.author, m.categories, m.keywords, m.input_type, m.input_data, m.output_data, m.version, m.external_id
Full Context:
[]

> Finished chain.

Observation: {'query': 'What is the ICICLE project?', 'result': "I'm sorry, but I don't have the information to answer that question."}
Thought: I should try using google_scholar to search for information about the ICICLE project.
Action: google_scholar
Action Input: ICICLE project
Observation: Title: ICICLE: Groupware for code inspection
Authors: M Muller
Summary: L Brothers, V Sembugamoorthy, M Muller - Proceedings of the 1990 …, 1990 - dl.acm.org
Total-Citations: 191

Title: ICICLE: Intelligent code inspection in a C language environmen

{'input': 'What is the ICICLE project?',
 'history': '',
 'output': 'The ICICLE project is a groupware for code inspection that was developed in the 1990s. It has been cited in various research papers and has a total of 191 citations on Google Scholar.'}

In [19]:
query="How many models were trained on the UCI Adult Dataset?"
agent(query)



> Entering new AgentExecutor chain...
 I should use ICICLE_MC to query the models trained on the UCI Adult Dataset.
Action: ICICLE_MC
Action Input: "How many models were trained on the UCI Adult Dataset?"

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:ModelCard)-[:TRAINED_ON]->(d:Datasheet)
WHERE d.name = "UCI Adult Dataset"
RETURN COUNT(m) as ModelsCount
Full Context:
[{'ModelsCount': 103}]

> Finished chain.

Observation: {'query': 'How many models were trained on the UCI Adult Dataset?', 'result': '103 models were trained on the UCI Adult Dataset.'}
Thought: I now know the final answer
Final Answer: 103 models were trained on the UCI Adult Dataset.

> Finished chain.


{'input': 'How many models were trained on the UCI Adult Dataset?',
 'history': 'Human: What is the ICICLE project?\nAI: The ICICLE project is a groupware for code inspection that was developed in the 1990s. It has been cited in various research papers and has a total of 191 citations on Google Scholar.',
 'output': '103 models were trained on the UCI Adult Dataset.'}

In [20]:
query="How many citations does the Model cards for model reporting paper have?"
agent(query)



> Entering new AgentExecutor chain...
 I should use google scholar to search for the paper and then use ICICLE_MC to query the database for the number of citations.
Action: google_scholar
Action Input: "GS: Model cards for model reporting"
Observation: Title: Model cards for model reporting
Authors: M Mitchell,A Zaldivar
Summary: M Mitchell, S Wu, A Zaldivar, P Barnes… - Proceedings of the …, 2019 - dl.acm.org
Total-Citations: 1621

Title: Low adherence to existing model reporting guidelines by commonly used clinical prediction models
Authors: A Callahan,BS Patel,KE Morse,D Dash
Summary: JH Lu, A Callahan, BS Patel, KE Morse, D Dash… - MedRXiv, 2021 - medrxiv.org
Total-Citations: 9

Title: Transparent Reporting of a multivariable prediction model for Individual Prognosis or Diagnosis (TRIPOD): explanation and elaboration
Authors: AJ Vickers,GS Collins
Summary: …, AJ Vickers, DF Ransohoff, GS Collins - Annals of internal …, 2015 - acpjournals.org
Total-Citations: 3724

Title: Is more 

{'input': 'How many citations does the Model cards for model reporting paper have?',
 'history': 'Human: What is the ICICLE project?\nAI: The ICICLE project is a groupware for code inspection that was developed in the 1990s. It has been cited in various research papers and has a total of 191 citations on Google Scholar.\nHuman: How many models were trained on the UCI Adult Dataset?\nAI: 103 models were trained on the UCI Adult Dataset.',
 'output': 'The Model cards for model reporting paper has a total of 1621 citations.'}

In [21]:
query="IMC: Which model in database that were trained on the UCI Adult Dataset, has the highest deployment mean accuracy on jetson-nano device?"
agent(query)



> Entering new AgentExecutor chain...
 I should use ICICLE_MC to query the database for the model with the highest deployment mean accuracy on the jetson-nano device.
Action: ICICLE_MC
Action Input: Which model in database that were trained on the UCI Adult Dataset, has the highest deployment mean accuracy on jetson-nano device?

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:ModelCard)-[:TRAINED_ON]->(d:Datasheet), (m)-[:Deployed_On]->(dep:Deployment), (dep)-[:On_Device_Type]->(ed:EdgeDevice)
WHERE d.name = "UCI Adult Dataset" AND ed.name = "jetson-nano"
RETURN m.name AS Model, dep.mean_accuracy AS Mean_Accuracy
ORDER BY dep.mean_accuracy DESC
LIMIT 1
Full Context:
[{'Model': 'UCI Adult Data Analysis', 'Mean_Accuracy': 0.9707495907289033}]

> Finished chain.

Observation: {'query': 'Which model in database that were trained on the UCI Adult Dataset, has the highest deployment mean accuracy on jetson-nano device?', 'result': 'The model that was trained on the U

{'input': 'IMC: Which model in database that were trained on the UCI Adult Dataset, has the highest deployment mean accuracy on jetson-nano device?',
 'history': 'Human: How many models were trained on the UCI Adult Dataset?\nAI: 103 models were trained on the UCI Adult Dataset.\nHuman: How many citations does the Model cards for model reporting paper have?\nAI: The Model cards for model reporting paper has a total of 1621 citations.',
 'output': 'The UCI Adult Data Analysis model has the highest deployment mean accuracy on the jetson-nano device.'}

In [22]:
query="Which paper has the highest amount of citations citing the UCI Adult Dataset?"
agent(query)



> Entering new AgentExecutor chain...
 I should use google scholar to search for papers related to the UCI Adult Dataset.
Action: google_scholar
Action Input: "GS: UCI Adult Dataset"
Observation: Title: Mitigating unwanted biases with adversarial learning
Authors: BH Zhang,M Mitchell
Summary: BH Zhang, B Lemoine, M Mitchell - Proceedings of the 2018 AAAI/ACM …, 2018 - dl.acm.org
Total-Citations: 1330

Title: Activeclean: Interactive data cleaning for statistical modeling
Authors: S Krishnan,J Wang,E Wu,MJ Franklin
Summary: S Krishnan, J Wang, E Wu, MJ Franklin… - Proceedings of the …, 2016 - dl.acm.org
Total-Citations: 281

Title: Improved autistic spectrum disorder estimation using Cfs subset with greedy stepwise feature selection technique
Authors: M Sharma
Summary: M Sharma - International Journal of Information Technology, 2022 - Springer
Total-Citations: 21

Title: Activeclean: Interactive data cleaning while learning convex loss models
Authors: S Krishnan,J Wang,E Wu,MJ Frankli

{'input': 'Which paper has the highest amount of citations citing the UCI Adult Dataset?',
 'history': 'Human: How many citations does the Model cards for model reporting paper have?\nAI: The Model cards for model reporting paper has a total of 1621 citations.\nHuman: IMC: Which model in database that were trained on the UCI Adult Dataset, has the highest deployment mean accuracy on jetson-nano device?\nAI: The UCI Adult Data Analysis model has the highest deployment mean accuracy on the jetson-nano device.',
 'output': 'The paper with the highest amount of citations citing the UCI Adult Dataset is "Mitigating unwanted biases with adversarial learning" by BH Zhang and M Mitchell with a total of 1330 citations.'}

In [23]:
query="Delete deployment with id 673b116f-b10e-4949-b21d-dd33c17ff3f8"
agent(query)



> Entering new AgentExecutor chain...
 I need to delete a deployment with the given id
Action: ICICLE_MC
Action Input: Delete deployment with id 673b116f-b10e-4949-b21d-dd33c17ff3f8

> Entering new GraphCypherQAChain chain...
Generated Cypher:
This is not allowed, contacting system administrator.


ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'h': expected 'e/E' (line 1, column 2 (offset: 1))
"This is not allowed, contacting system administrator."
  ^}

In [24]:
query="Which model and version has the best power consumption for chicago-03-city?"
agent(query)



> Entering new AgentExecutor chain...
 I should first check the ICICLE_MC database for information on models and versions.
Action: ICICLE_MC
Action Input: "Which model and version has the best power consumption for chicago-03-city?"

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:ModelCard)-[:Deployed_On]->(d:Deployment)
WHERE d.deployment_location = 'chicago-03-city'
RETURN m.name AS Model, m.version AS Version, d.power_consumption_average_watts AS Power_Consumption
ORDER BY d.power_consumption_average_watts ASC
LIMIT 1
Full Context:
[{'Model': 'UCI Adult Data Analysis', 'Version': '0.1', 'Power_Consumption': 25}]

> Finished chain.

Observation: {'query': 'Which model and version has the best power consumption for chicago-03-city?', 'result': 'The model UCI Adult Data Analysis with version 0.1 has the best power consumption for chicago-03-city.'}
Thought: This result seems specific to a particular dataset. I should also check Google Scholar for any relevant r

{'input': 'Which model and version has the best power consumption for chicago-03-city?',
 'history': 'Human: IMC: Which model in database that were trained on the UCI Adult Dataset, has the highest deployment mean accuracy on jetson-nano device?\nAI: The UCI Adult Data Analysis model has the highest deployment mean accuracy on the jetson-nano device.\nHuman: Which paper has the highest amount of citations citing the UCI Adult Dataset?\nAI: The paper with the highest amount of citations citing the UCI Adult Dataset is "Mitigating unwanted biases with adversarial learning" by BH Zhang and M Mitchell with a total of 1330 citations.',
 'output': "Without specific information about the context or product category (like cars, computers, HVAC systems, etc.), it's impossible to provide an accurate answer to this question. Please provide more details."}

In [25]:
query="How many models are in the database?"
response = agent(query)
print(response['output'])



> Entering new AgentExecutor chain...
 I should use ICICLE_MC to query the database for models.
Action: ICICLE_MC
Action Input: "How many models are in the database?"

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:AIModel)
RETURN COUNT(m) as NumberOfModels
Full Context:
[{'NumberOfModels': 104}]

> Finished chain.

Observation: {'query': 'How many models are in the database?', 'result': 'There are 104 models in the database.'}
Thought: I now know the final answer
Final Answer: There are 104 models in the database.

> Finished chain.
There are 104 models in the database.


In [47]:
from langchain_community.llms import Ollama

llm = Ollama(model="gemma")

llm.invoke("Tell me a joke")

'Why did the tomato turn red?\n\nBecause it saw the salad dressing.'